In [1]:
from pathlib import Path
from os import urandom, getcwd
from datetime import datetime
from copy import deepcopy
from json import dump

from pandas import read_excel
from numpy import isnan
from sklearn.preprocessing import StandardScaler
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from IPython.display import Image, display

from machine_learning import Grid, HyperTune
from machine_learning.misc import zip_run_name_files
from machine_learning.featurization import featurize_si_aerogels

In [2]:
dataset = r"machine_learning_si_aerogels.csv"
folder = "si_aerogels"
file_path = "files/si_aerogels/machine_learning_si_aerogels.xlsx/"

data_path = str(Path(getcwd()) / file_path)
data = read_excel(data_path)
seed = int.from_bytes(urandom(3), "big")  # Generate an actual random number
algorithm = 'nn'

now = datetime.now()
date_string = now.strftime("_%Y%m%d-%H%M%S")
run_name = f"LatentSpace_{algorithm}_{date_string}_{seed}"

y_column = 'Surface Area m2/g'
drop_columns = ['Title', 'Porosity', 'Porosity %', 'Pore Volume cm3/g', 'Average Pore Diameter nm',
                'Bulk Density g/cm3', 'Young Modulus MPa', 'Crystalline Phase',
                'Average Pore Size nm', 'Thermal Conductivity W/mK', 'Gelation Time mins']
data

,Title,Si Precursor,Si Precursor Concentration M,Co-Precursor Concentration M,Dopant Concentration M,Solvent 1,Solvent 1 Concentration M,Solvent 2,Solvent 2 Concentration M,Additional Solvents,...,Wash Solvent 3 2,Wash Solvent 3 3,Wash Solvent 4 0,Wash Solvent 4 1,Wash Solvent 4 2,Wash Solvent 4 3,Wash Solvent 5 0,Wash Solvent 5 1,Drying Solvent 0,Drying Solvent 1
0,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,12.20,H2O,22.43,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
1,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,12.20,H2O,22.43,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
2,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,8.02,H2O,16.31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
3,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,8.02,H2O,16.31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
4,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0071,NaN,MeOH,7.99,H2O,16.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN
945,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN
946,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN
947,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN


In [3]:
def filter_papers(df):
    df = df.loc[df['Final Gel Type'] == 'Aerogel']

    unique_titles = set(df['Title'].tolist())
    papers_with_unknown_sa = 0
    papers_with_all_unknown_sa = 0
    number_of_aerogels_to_fill = 0
    bad_titles = []  # Titles to remove because all aerogels have unknown surface area

    for unique_title in unique_titles:
        sub_df = df.loc[df['Title'] == unique_title]  # Grab rows with title
        surface_areas = sub_df['Surface Area m2/g'].to_numpy()  # Cast the surface areas to a numpy array
        nan_sa = surface_areas[isnan(surface_areas)]  # Grab all nan values in the surface areas array

        if any(nan_sa):
            papers_with_unknown_sa += 1

        if len(nan_sa) == len(surface_areas):  # If entire numpy array is nan
            papers_with_all_unknown_sa += 1
            bad_titles.append(unique_title)

        elif any(nan_sa):  # If there are nan in array, but the entire array is not nan
            number_of_aerogels_to_fill += len(nan_sa)

    for bad_title in bad_titles:
        df = df.loc[df['Title'] != bad_title]

    general_info = {'Number of unique titles that contain aerogels': len(unique_titles),
                    'Number of papers with unknown surface areas': papers_with_unknown_sa,
                    'Number of papers with all unknown surface areas': papers_with_all_unknown_sa,
                    'Number of aerogels to predict surface area': number_of_aerogels_to_fill,
                    'Titles to be removed with Aerogels because all surface areas are unknown': bad_titles,
                    }

    return df, general_info

In [4]:
all_data = deepcopy(data)  # Copy orginial data
all_data = all_data.reset_index()  # Add column with indexes

# Remove any papers that have all nan surface area aerogels
all_data, general_info = filter_papers(all_data)
all_data

,index,Title,Si Precursor,Si Precursor Concentration M,Co-Precursor Concentration M,Dopant Concentration M,Solvent 1,Solvent 1 Concentration M,Solvent 2,Solvent 2 Concentration M,...,Wash Solvent 3 2,Wash Solvent 3 3,Wash Solvent 4 0,Wash Solvent 4 1,Wash Solvent 4 2,Wash Solvent 4 3,Wash Solvent 5 0,Wash Solvent 5 1,Drying Solvent 0,Drying Solvent 1
0,0,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,12.20,H2O,22.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
1,1,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,12.20,H2O,22.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
2,2,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,8.02,H2O,16.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
3,3,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0043,NaN,MeOH,8.02,H2O,16.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
4,4,Novel approach of silica-PVA hybrid aerogel sy...,TMOS,0.8,0.0071,NaN,MeOH,7.99,H2O,16.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,944,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN
945,945,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN
946,946,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN
947,947,Novel Al2O3–SiO2 composite aerogels with high ...,TEOS,NaN,NaN,NaN,EtOH,NaN,H2O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EtOH,NaN


In [5]:
# Gather and featurize all data, dropping xerogels in the process
all_data = featurize_si_aerogels(df=all_data, str_method="one_hot_encode", num_method="smart_values",
                                 y_columns=[y_column], drop_columns=drop_columns, remove_xerogels=True,
                                 drop_rows_missing_y=False, leave_index=True)
all_data

,index,Si Precursor Concentration M,Co-Precursor Concentration M,Dopant Concentration M,Solvent 1 Concentration M,Solvent 2 Concentration M,Acid Catalyst Initial Concentration M,Acid Catalyst concentration in SolM,Base Catalyst Initial ConcentrationM,Base Catalyst concentration in Sol M,...,Wash Solvent 5 1_NaN,Drying Solvent 0_CO2,Drying Solvent 0_EtOH,Drying Solvent 0_HFIP,Drying Solvent 0_Isopropanol,Drying Solvent 0_MTBE,Drying Solvent 0_NaN,Drying Solvent 0_Nitrogen Atmosphere,Drying Solvent 1_MTMS,Drying Solvent 1_NaN
0,0,0.8,0.0043,0.0,12.20,22.43,0.0,0.0,0.0,0.00699,...,1,1,0,0,0,0,0,0,0,1
1,1,0.8,0.0043,0.0,12.20,22.43,0.0,0.0,0.0,0.00000,...,1,1,0,0,0,0,0,0,0,1
2,2,0.8,0.0043,0.0,8.02,16.31,0.0,0.0,0.0,0.00699,...,1,1,0,0,0,0,0,0,0,1
3,3,0.8,0.0043,0.0,8.02,16.31,0.0,0.0,0.0,0.00000,...,1,1,0,0,0,0,0,0,0,1
4,4,0.8,0.0071,0.0,7.99,16.25,0.0,0.0,0.0,0.00696,...,1,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,944,0.0,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.00000,...,1,0,1,0,0,0,0,0,0,1
945,945,0.0,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.00000,...,1,0,1,0,0,0,0,0,0,1
946,946,0.0,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.00000,...,1,0,1,0,0,0,0,0,0,1
947,947,0.0,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.00000,...,1,0,1,0,0,0,0,0,0,1


In [6]:
# Grab all rows where a surface area exists
training_data = all_data.loc[isnan(all_data[y_column]) == False]
training_data = training_data.drop('index', axis=1)

# Grab all rows where a surface area does not exists
data_to_predict = all_data.loc[isnan(all_data[y_column]) == True]
predicting_indexes = data_to_predict['index']
data_to_predict = data_to_predict.drop('index', axis=1)
data_to_predict = data_to_predict.drop(y_column, axis=1)

In [7]:
def model(training_data, testing_data, validation_percent: float = 0.1):

    # Create training_features
    train_features = training_data.drop(y_column, axis=1)
    train_target = training_data[y_column]

    # Define testing features
    test_features = testing_data

    # Create scalers
    features_scaler = StandardScaler()
    target_scaler = StandardScaler()

    # Scale training/testing features
    train_features = features_scaler.fit_transform(train_features)
    test_features = features_scaler.transform(test_features)

    # Scale train target
    train_target = target_scaler.fit_transform(train_target.to_numpy().reshape(-1, 1)).reshape(-1, )

    # Find HyperParameters
    grid = Grid.make_normal_grid(algorithm)  # Make grid for hyper tuning based on algorithm
    tuner = HyperTune(algorithm, train_features, train_target, grid, opt_iter=10,
                      cv_folds=10)  # Get parameters for hyper tuning

    tuner, estimator, params = tuner.hyper_tune(method="random")  # Hyper tuning the model
    tuner.plot_overfit(run_name=run_name)
    tuner.plot_val_pva(run_name=run_name)

    estimator.fit(train_features, train_target, epochs=20)
    predictions = estimator.predict(test_features)

    # Unscale the predictions
    predictions = target_scaler.inverse_transform(predictions.reshape(-1, 1)).reshape(-1, )
    return predictions, tuner

In [ ]:
predictions,tuner = model(training_data, data_to_predict, validation_percent=0.1)

Trial 27 Complete [00h 00m 03s]
val_loss: 0.4900256395339966

Best val_loss So Far: 0.37949296832084656
Total elapsed time: 00h 00m 44s

Search: Running Trial #28

Hyperparameter    |Value             |Best Value So Far 
units             |256               |312               
learning_rate     |0.002             |0.002             
tuner/epochs      |20                |7                 
tuner/initial_e...|0                 |3                 
tuner/bracket     |0                 |2                 
tuner/round       |0                 |1                 

Epoch 1/20
 7/21 [=========>....................] - ETA: 0s - loss: 2.0923 - mse: 2.0923 - mae: 1.1384

2021-07-14 17:15:13.862821: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-07-14 17:15:13.862852: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-07-14 17:15:13.879520: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-07-14 17:15:13.880303: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-07-14 17:15:13.881175: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: untitled_project/3484c97087d0609c98ca357adfdcd34b/execution0/train/plugins/profile/2021_07_14_17_15_13
2021-07-14 17:15:13.881701: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to untitled_project/3484c97087d0609c98ca357adfdcd34b/execution0/train/plugins/profile/2021_07_14_17_15_13/user-Desktop.trace.json.gz
2021-07-14 17:15:13.882715: I tensorflow/core/profiler/rpc/client/save_pr

In [ ]:
predictions

In [ ]:
tuner.plot_overfit(run_name=run_name)
overfit_path = Path(getcwd()) / f"{run_name}_overfit.png"
display(Image(filename=overfit_path)) 

In [ ]:
tuner.plot_val_pva(run_name=run_name)
val_pva_path = Path(getcwd()) / f"{run_name}_PVA.png"
display(Image(filename=val_pva_path)) 

In [ ]:
def save_predictions(predicted_data):

    raw_data_path = "files/si_aerogels/raw_si_aerogels.xlsx/"
    raw_data_path = str(Path(getcwd()) / raw_data_path)
    wb = load_workbook(filename=raw_data_path)

    for _, row in predicted_data.iterrows():

        # The DataFrame start index at 0, whereas Excel starts at 2 (considering the header also counts as a row)
        # There is an additional header row with specifying info, so add one for that as well
        index = int(row['index']) + 3
        sa = row['Surface Area m2/g']

        # Fill in cell value with calculated surface area
        wb['Comprehensive'][f"GZ{index}"] = sa

        # Color the row to let user know what rows were predicted
        for cell in wb['Comprehensive'][f"A{index}":f"HF{index}"][0]:
            cell.fill = PatternFill("solid", fgColor="fff000")

    output_file = Path(getcwd()) / f"{run_name}_si_aerogels_with_predicted.xlsx"
    wb.save(filename=output_file)

In [ ]:
data_to_predict[y_column] = predictions
data_to_predict['index'] = predicting_indexes
save_predictions(data_to_predict)